In [ ]:
import os
import glob
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
from nltk.stem.porter import PorterStemmer
!pip install autocorrect
from autocorrect import Speller
from collections import OrderedDict 
from pathlib import Path
import shutil
import spacy 
import re
nltk.download('wordnet')
nltk.download('stopwords')
!pip install contractions
!pip install spacy
!pip install textblob
import contractions
from textblob import TextBlob, Word
import pandas as pd
import numpy as np
import ast

In [10]:
paths = []
filename = []
# /content/drive/MyDrive/stories
# /content/drive/MyDrive/IR ASSIGNMENT 2/stories
for (dirpath, dirnames, filenames) in os.walk("/content/drive/MyDrive/stories"):
  for i in filenames:
    paths.append(str(dirpath)+str("/")+i)
    filename.append(i)


In [11]:
len(paths)

467

In [12]:
documentsDict = {}
# filenameOfDoc = {}
# global counter 
# counter = 0
def cleaning(filepath,filename,flag):
  if(flag==1):
    f = open(filepath, 'r', errors = 'ignore',encoding='cp1250').read().strip()
    clean_S = contractions.fix(f)
    clean_S=re.sub("([\-]{1,2})", " ",clean_S)
    clean_s=re.sub("[\,]", " ",clean_S)
    clean_S = re.sub(r'[\!@#$%^&\*()\_+={}\:\;<>\?/\|\-/"\']*',"", clean_S)  #junk symbols
    # clean_S = re.sub(r"(.)\1{2}","",clean_S)  #for elongated words
    clean_S=re.sub("([\.\!]{2})", " ",clean_S)  #removing punctuations
    clean_S = re.sub('\s+', ' ', clean_S)    
    clean_S=re.sub("[^a-zA-Z\s\n]", " ",clean_S)  #used to eliminate non-ascii and numeric symbols 
    clean_S = re.sub(r"(.)\1{2}","",clean_S)  #for elongated words
    clean_S = clean_S.lower()
    documentsDict[filename]=clean_S
    # counter = counter+1
    # filenameOfDoc[counter] = filename

  elif(flag==0):
    filepath = contractions.fix(filepath)
    filepath=re.sub("([\-]{1,2})", " ",filepath)
    filepath=re.sub("[0-9]", "",filepath)
    # filepath=re.sub("[\,]", " ",filepath)
    filepath = re.sub(r'[\!@#$%^&\*()\_+={}\:\;<>\?/\|\-/"\']*',"", filepath)
    # filepath = re.sub(r"(.)\1{2}","",filepath)  #for elongated words
    filepath=re.sub("([\.\!]{2})", " ",filepath)
    filepath = re.sub('\s+', ' ', filepath)
    filepath=re.sub("[^a-zA-Z\s\n]", " ",filepath)
    filepath = re.sub(r"(.)\1{2}","",filepath)  #for elongated words
    filepath = filepath.lower()
    return filepath

In [13]:
for i in range(0,len(filename)):
       cleaning(paths[i],filename[i],1)

In [14]:
len(documentsDict)

467

In [15]:
f = open('cleaned467.txt', 'w')
f.write(str(documentsDict))
f.close()

In [ ]:
documentsDict

In [16]:
def gen_token(flag,query):
  tokenizer = RegexpTokenizer(r'\w+')
  if(flag==1):
    for docname in documentsDict:
        documentsDict[docname] = tokenizer.tokenize(str(documentsDict[docname]))

  elif(flag==0):
    query = tokenizer.tokenize(query)
    return query

In [17]:
def remove_stopword(flag,query):
  stopwordsList = stopwords.words('english')
  if(flag==1):
    for docname in documentsDict:
        for word in documentsDict[docname]:
            if word in stopwordsList:
                documentsDict[docname].remove(word)
  elif(flag==0):
      q=[]
      for word in query:
            if word not in stopwordsList:
              q.append(word)
      query=q.copy()
      return query

In [18]:
def normalise_using_textblob(flag,query):
  if(flag==1):       
    for docname in documentsDict:
        dataList = []
        for word in documentsDict[docname]:
            w = Word(word)
            dataList.append(w.lemmatize())
        documentsDict[docname] = dataList   
  elif(flag==0):
    ans = []
    for word in query:
      w=Word(word)
      ans.append(w.lemmatize())
    return ans

In [44]:
def proquery(query):
  queryFinal=[]
  query = cleaning(query," ",0)
  queryFinal = gen_token(0,query)
  queryFinal = remove_stopword(0,queryFinal)
  queryFinal = normalise_using_textblob(0,queryFinal)
  #queryFinal = queryFinal.replace(","," ").split()
  return queryFinal

In [19]:
#preprocessing DocumentDict after Cleaning
gen_token(1,"")
remove_stopword(1,"")
normalise_using_textblob(1,"")

In [20]:
len(documentsDict)

467

In [46]:
documentsFilename = list(documentsDict.keys())

In [48]:
documentsFilename
f = open("documentsFilenameLem.txt","w")
f.write(str(documentsFilename))
f.close()

In [63]:
f = open("documentsFilenameLem.txt","r")
x = f.read()
x = x.replace("'","")
x = x.replace("[","")
x = x.replace("]","")
documentsFilename = x.split(",")
f.close()


Positional List

In [24]:
Postings = pd.DataFrame()

Final Postings

In [25]:
for docid in range(0,len(documentsFilename)):
  tokensOfDocid = documentsDict[documentsFilename[docid]]
  for position in range(0,len(tokensOfDocid)):
    word = tokensOfDocid[position]
    if word in Postings:
      if docid in Postings.loc[1][word]:
        Postings.loc[1][word][docid].append(position)
      else:
        Postings.iloc[1][word][docid] = [position]
        Postings.iloc[0][word] = Postings.iloc[0][word] + 1
    else:
      Postings.insert(value=[1,{docid:[position]}], loc=0, column=word)

In [26]:
Postings

suborner  ...                                               give
0               1  ...                                                245
1  {466: [65626]}  ...  {0: [0], 2: [2790], 4: [474, 803, 1443], 5: [2...

[2 rows x 50523 columns]

In [27]:
Postings.to_csv("/content/drive/MyDrive/IR ASSIGNMENT 2/PostingsLem.csv")

Support for searching

In [28]:
Postings = pd.read_csv("/content/drive/MyDrive/IR ASSIGNMENT 2/PostingsLem.csv")

In [29]:
Postings = Postings.drop(["Unnamed: 0"],axis=1)

In [30]:
Postings

suborner  ...                                               give
0               1  ...                                                245
1  {466: [65626]}  ...  {0: [0], 2: [2790], 4: [474, 803, 1443], 5: [2...

[2 rows x 50523 columns]

In [31]:
for column in Postings.columns:
  xa = ast.literal_eval(Postings.iloc[1][column])
  Postings.iloc[1][column] = xa

In [ ]:
f = open("documentsFilenameLem.txt","r")
x = f.read()
x = x.replace("'","")
x = x.replace("[","")
x = x.replace("]","")
documentsFilename = x.split(",")
f.close()

In [66]:
query = "two days"
#do preprocessing
tokenquery = proquery(query) #['to', 'be', 'or', 'not']
#get machlist at each bi-iteration
tokenPostingFirst = {}
tempPosting = {}
for token in range(0,len(tokenquery)-1):
  if tokenquery[token] in Postings:
    if tokenPostingFirst == {}:
      tokenPostingFirst = Postings[tokenquery[token]][1]

    else:
      tokenPostingFirst = tempPosting
      tempPosting = {}
    #for nexttoken in range(token,len(tokenquery)):
    if tokenquery[token+1] in Postings: 
      tokenPostingsecond = Postings[tokenquery[token +1]][1]
      #checking same doc or not from tokenpostingfirst and second
      for doc_id in tokenPostingFirst:
        if doc_id in tokenPostingsecond:
          #now checking positions of tokenfirst in docid consequtive or not with tokensecond
          iteration = tokenPostingFirst[doc_id]
          # print(str(type(iteration))+" docid "+str(doc_id)+" token "+str(token))
          # print(iteration)
          for positionOftoken in iteration:
            if positionOftoken+1 in tokenPostingsecond[doc_id]:
              if doc_id in tempPosting:
                tempPosting[doc_id].append(positionOftoken +1)
              else:
                tempPosting[doc_id] = []
                tempPosting[doc_id].append(positionOftoken + 1)
  else:
    break
print("Entered Query : "+ str(tokenquery))
print("Number of Documents Retrieved : "+ str(len(tempPosting.keys())))
print("Documents containing query are following : ")
for docid in tempPosting.keys():
  print(documentsFilename[docid])

Entered Query : ['two', 'day']
Number of Documents Retrieved : 46
Documents containing query are following : 
 abbey.txt
 hitch3.txt
 cardcnt.txt
 psi
 valen
 sre04.txt
 keepmodu.txt
 forgotte
 fea2
 hitch2.txt
 dakota.txt
 alad10.txt
 history5.txt
 bureau.txt
 hound-b.txt
 musgrave.txt
 timem.hac
 quest
 fleas.txt
 testpilo.hum
 blackp.txt
 6napolen.txt
 3lpigs.txt
 consumdr.hum
 ab40thv.txt
 roger1.txt
 imagin.hum
 robotech
 7voysinb.txt
 veiledl.txt
 wisteria.txt
 5orange.txt
 cybersla.txt
 beggars.txt
 floc
 reap
 girlclub.txt
 vgilante.txt
 fgoose.txt
 zombies.txt
 ezoff
 darkness.txt
 dskool.txt
 hellmach.txt
 solitary.txt
 gulliver.txt
